In [3]:
import pandas as pd
full_df = pd.read_csv('test-v7.csv')

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import euclidean_distances

import nltk
from nltk.tag import pos_tag
from nltk import word_tokenize
from nltk.corpus import stopwords
stop = stopwords.words('english')
def cleaning_text(sentence):
    sentence = str(sentence)
    sentence=sentence.lower()
    sentence=re.sub('[^\w\s]',' ', sentence) #removes punctuations
    sentence=re.sub('_',' ', sentence) #removes punctuations
    sentence=re.sub('\d+',' ', sentence) #removes digits
    cleaned=' '.join([w for w in sentence.split() if not w in stop]) # removes english stopwords
    cleaned=' '.join([w for w , pos in pos_tag(cleaned.split()) if (pos == 'NN' or pos=='JJ' or pos=='JJR' or pos=='JJS' )])
    
#selecting only nouns and adjectives
    cleaned=' '.join([w for w in cleaned.split() if not len(w)<=2 ]) #removes single lettered words and digits
    cleaned=cleaned.strip()
    return cleaned
	  
full_df['cleaned'] = full_df['description'].apply(lambda x: cleaning_text(x))

In [5]:
full_df.count()

aptID                   74659
bathrooms               74659
bedrooms                74659
building_id             62281
created                 74659
description             69434
display_address         74434
latitude                74659
longitude               74659
listing_id              74659
manager_id              74659
price                   74659
street_address          74646
created.Day             74659
created.Month           74659
created.Year            74659
created.Date            74659
created.WDay            74659
created.WDayLbl         74659
created.Week            74659
created.Hour            74659
created.Yday            74659
created.MWeek           74659
photoCount              74659
featureCount            74659
dining_room             74659
pre_war                 74659
laundry_in_building     74659
dishwasher              74659
hardwood_floors         74659
dogs                    74659
cats                    74659
doorman                 74659
elevator  

In [6]:
trimDF_test = full_df[['aptID','cleaned']]
trimDF_test.to_csv('trimDF_test_v7_200.csv')

In [7]:
from gensim.models import doc2vec
import nltk
import re

def split_sentence(sentence):
    words = re.split('\W+', sentence.lower())
    return [word for word in words if word != ""]

class MyDocs(object):
    def __iter__(self):
        for i, text in enumerate(open("trimDF_test_v7_200.csv")):
            if i:
                yield doc2vec.LabeledSentence(words=split_sentence(text), tags=['%s' % i])

# Train the doc2vec model
mydocs = MyDocs()
model = doc2vec.Doc2Vec(mydocs, size = 200, window = 8, min_count = 5, workers = 4)
model.save('description.model')

In [8]:
test = pd.read_csv('trimDF_test_v7_200.csv')
test['docvecs'] = model.docvecs

In [11]:
test.head()

,Unnamed: 0,aptID,cleaned,docvecs
0,0,0,large awesome terrace accessible bedroom livin...,"[-0.0355756, 0.0739287, -0.0331795, 0.0210779,..."
1,1,1,prime soho bleecker houston stainless steel ma...,"[0.00441408, 0.0191517, 0.00314035, 0.00483735..."
2,2,2,spacious studio prime location management supe...,"[-0.0286744, 0.0285134, -0.00368225, 0.0045760..."
3,3,3,immediate access call bryan bond new york real...,"[0.0034241, 0.0333762, 0.0055365, 0.0135253, 0..."
4,4,5,beautiful true bedroom luxury financial distri...,"[0.0119787, 0.0165289, -0.0147353, -0.0135618,..."


In [32]:
test['docvecs'].apply(pd.Series).add_prefix('value')

,value0,value1,value2,value3,value4,value5,value6,value7,value8,value9,...,value190,value191,value192,value193,value194,value195,value196,value197,value198,value199
0,-0.035576,0.073929,-0.033180,0.021078,0.000428,-0.092205,0.054964,-0.098008,-0.023262,-0.052372,...,-0.021965,0.022739,0.023465,-0.004721,0.032670,0.021266,0.096344,-0.035579,-0.003249,0.062437
1,0.004414,0.019152,0.003140,0.004837,-0.005812,-0.022263,0.025462,-0.056201,0.025206,-0.011649,...,0.003185,0.003495,-0.005786,-0.030693,0.024203,0.003275,0.010028,-0.025118,-0.010128,0.006000
2,-0.028674,0.028513,-0.003682,0.004576,-0.020499,0.013403,0.075614,-0.017976,0.007437,-0.040283,...,0.020060,-0.009825,-0.027832,-0.009368,0.058619,-0.023031,0.013396,-0.011196,0.010921,0.011746
3,0.003424,0.033376,0.005537,0.013525,0.001937,-0.017018,0.028434,-0.037660,0.008523,-0.014063,...,-0.015970,-0.005637,-0.017501,-0.019858,0.008968,0.010088,0.017267,-0.030390,-0.007797,0.013852
4,0.011979,0.016529,-0.014735,-0.013562,-0.055827,-0.056112,0.057593,-0.038702,-0.041710,0.001676,...,-0.002763,-0.043709,0.007733,-0.022064,0.009126,0.008446,0.006115,0.006116,0.028430,0.010808
5,0.044518,0.041109,-0.020683,-0.037038,0.020900,-0.043240,0.049353,-0.095571,-0.074882,-0.051899,...,-0.030982,-0.009547,-0.022166,-0.109625,0.009997,-0.006858,0.003020,-0.052512,-0.014906,-0.005770
6,-0.009045,0.012393,0.000537,0.006146,0.011072,-0.010854,0.015041,-0.015842,0.002336,-0.006337,...,-0.015649,-0.001531,-0.011079,-0.000045,0.012730,0.005816,0.010631,-0.018690,-0.006953,0.009896
7,0.024061,0.053716,0.013486,0.035058,0.007272,-0.013872,0.026848,-0.054740,-0.026297,0.008407,...,-0.020451,0.028124,-0.025467,-0.000573,0.011030,0.009485,0.024092,-0.025459,-0.025594,-0.001227
8,-0.009511,0.023243,0.000182,0.003618,0.003187,-0.015193,0.020856,-0.023783,0.012918,-0.007825,...,-0.010638,0.001528,0.001167,0.002324,0.005458,0.014171,0.003982,-0.019590,-0.010842,0.007182
9,-0.011572,0.018009,-0.007391,0.010860,-0.005535,-0.020238,0.011921,-0.028956,0.010626,-0.014788,...,-0.013313,-0.001114,-0.008287,0.007180,0.003879,0.001863,-0.000718,-0.019409,-0.009436,0.010252


In [34]:
test = pd.concat([test.drop(['docvecs'], axis=1), test['docvecs'].apply(pd.Series).add_prefix('value')], axis=1)

In [35]:
test.head()

,Unnamed: 0,aptID,cleaned,value0,value1,value2,value3,value4,value5,value6,...,value190,value191,value192,value193,value194,value195,value196,value197,value198,value199
0,0,0,large awesome terrace accessible bedroom livin...,-0.035576,0.073929,-0.033180,0.021078,0.000428,-0.092205,0.054964,...,-0.021965,0.022739,0.023465,-0.004721,0.032670,0.021266,0.096344,-0.035579,-0.003249,0.062437
1,1,1,prime soho bleecker houston stainless steel ma...,0.004414,0.019152,0.003140,0.004837,-0.005812,-0.022263,0.025462,...,0.003185,0.003495,-0.005786,-0.030693,0.024203,0.003275,0.010028,-0.025118,-0.010128,0.006000
2,2,2,spacious studio prime location management supe...,-0.028674,0.028513,-0.003682,0.004576,-0.020499,0.013403,0.075614,...,0.020060,-0.009825,-0.027832,-0.009368,0.058619,-0.023031,0.013396,-0.011196,0.010921,0.011746
3,3,3,immediate access call bryan bond new york real...,0.003424,0.033376,0.005537,0.013525,0.001937,-0.017018,0.028434,...,-0.015970,-0.005637,-0.017501,-0.019858,0.008968,0.010088,0.017267,-0.030390,-0.007797,0.013852
4,4,5,beautiful true bedroom luxury financial distri...,0.011979,0.016529,-0.014735,-0.013562,-0.055827,-0.056112,0.057593,...,-0.002763,-0.043709,0.007733,-0.022064,0.009126,0.008446,0.006115,0.006116,0.028430,0.010808


In [36]:
test.to_csv('description_vec_test_v7_200.csv', index =False)